# Compliance

Conformità della popolazione alle norme di igiene e sicurezza.

> Max Pierini

***

# Introduzione

In questa delicata fase, la maggior parte delle attività ha ripreso (o riprenderà presto) il consuento ritmo e la popolazione è libera di circolare sul territorio nazionale e, in parte, internazionale.

Il contenimento quindi dell'epidemia dipende in gran parte, in assenza di un vaccino, dalla conformità della popolazione alle norme di igiene e sicurezza, al corretto uso dei Dispositivi di Protezione Indiviuale (**DPI**, mascherine, guanti dove indicato, ecc) e di distanziamento sociale, imposte e/o suggerite dal Ministero della Salute dall'Istituto Superiore di Sanità.

Come sta rispondendo l'Italia? Gli italiani si attengono alle norme?

YouGov e l'Imperial College pubblicano regolarmente i risultati di un utile sondaggio internazionale sull'analisi del comportamento in fase pandemica, aggiornato con cadenza settimanale ([YouGov](https://github.com/YouGov-Data/covid-19-tracker)).

In [1]:
from copy import deepcopy
import datetime as dt

import numpy as np
import pandas as pd
import scipy.stats as sps
import matplotlib.pyplot as plt
from matplotlib import dates as mdates
from matplotlib import ticker

from IPython.display import display, Markdown, Latex

In [2]:
import seaborn as sns
sns.set()
sns.set_palette('colorblind')

In [3]:
nomimal_columns = [
    "i12_health_1",   # Worn mask outside
    "i12_health_2",   # Washed hands
    "i12_health_3",   # Used hand sanitiser
    "i12_health_4",   # Covered your nose and mouth when sneezing or coughing
    "i12_health_5",   # Avoided contact with people who have symptoms
    "i12_health_6",   # Avoided going out in general
    "i12_health_7",   # Avoided going to hospital or other
    "i12_health_8",   # Avoided taking public transport
    "i12_health_9",   # Avoided working outside your home
    "i12_health_10",  # Avoided letting your children go to school
    "i12_health_11",  # Avoided having guests to your home
    "i12_health_12",  # Avoided small social gatherings
    "i12_health_13",  # Avoided medium-sized social gatherings
    "i12_health_14",  # Avoided large-sized social gatherings
    "i12_health_15",  # Avoided crowded areas
    "i12_health_16",  # Avoided going to shops
    "i12_health_17",  # Slept in separate bedrooms
    "i12_health_18",  # Eaten separately at home
    "i12_health_19",  # Cleaned frequently touched surfaces
    "i12_health_20",  # Avoided touching objects in public
]

baselines = {
    # Common behaviours
    "i12_health_2":  2,  # Washed hands
    "i12_health_3":  2,  # Used hand sanitiser
    "i12_health_4":  2,  # Covered your nose and mouth when sneezing or coughing
    "i12_health_5":  2,  # Avoided contact with people who have symptoms
    "i12_health_8":  2,  # Avoided taking public transport
    # Specific behaviours
    "i12_health_1":  0,  # Worn mask outside
    "i12_health_6":  0,  # Avoided going out in general
    "i12_health_7":  0,  # Avoided going to hospital or other
    "i12_health_9":  0,  # Avoided working outside your home
    "i12_health_11": 0,  # Avoided having guests to your home
    "i12_health_12": 0,  # Avoided small social gatherings
    "i12_health_13": 0,  # Avoided medium-sized social gatherings
    "i12_health_14": 0,  # Avoided large-sized social gatherings
    "i12_health_15": 0,  # Avoided crowded areas
    "i12_health_16": 0,  # Avoided going to shops
    "i12_health_17": 0,  # Slept in separate bedrooms
    "i12_health_18": 0,  # Eaten separately at home
    "i12_health_19": 0,  # Cleaned frequently touched surfaces
    "i12_health_20": 0,  # Avoided touching objects in public
}

nomimal_columns_dict = {
    "i12_health_1": "Indossare la mascherina all'esterno",
    "i12_health_2": "Lavarsi frequentemente le mani",
    "i12_health_3": "Usare gel igienizzanti",
    "i12_health_4": "Coprirsi naso e bocca se starnuti o tosse",
    "i12_health_5": "Evitare contatti con persone sintomatiche",
    "i12_health_6": "Evitare di uscire di casa in generale",
    "i12_health_7": "Evitare di andare in ospedali e ambulatori",
    "i12_health_8": "Evitare di usare i trasporti pubblici",
    "i12_health_9": "Evitare di lavorare fuori casa",
    "i12_health_10": "Evitare ai figli di andare a scuole e corsi",
    "i12_health_11": "Evitare di avere ospiti a casa",
    "i12_health_12": "Evitare piccoli assembramenti",
    "i12_health_13": "Evitare medi assembramenti",
    "i12_health_14": "Evitare grandi assembramenti",
    "i12_health_15": "Evitare zone affollate",
    "i12_health_16": "Evitare di frequentare negozi",
    "i12_health_17": "Dormire in stanze separate",
    "i12_health_18": "Mangiare in aree diverse in casa",
    "i12_health_19": "Pulire spesso le superfici toccate",
    "i12_health_20": "Evitare di toccare oggetti in pubblico",
}

# NOT TO BE IMPLEMENTED YET
value_colums = [
    "i13_health",  # how many times, would you say you washed your hands
]

nominal_to_value = {
    "Not at all":  0,  # Worst behaviour
    "Rarely" :    .25,
    "Sometimes":  .5,
    "Frequently": .75,
    "Always":      1,  # Best behaviour
}

valori_ita = ["Mai", "Raramente", "Qualche volta", "Spesso", "Sempre"]

region_ita = {
    "Centre": "Centro",
    "Islands": "Isole",
    "North east": "Nord Est",
    "North west": "Nord Ovest",
    "South": "Sud",
}

zone_regioni = {
"North west": ["Aosta","Liguria","Lombardy","Piedmont"],
"North east": ["Emilia-Romagna","Friuli-Venezia_Giulia","Trento","Bolzano","Veneto"],
"Centre": ["Lazio","Marche","Tuscany","Umbria"],
"South": ["Abruzzo","Apulia","Basilicata","Calabria","Campania","Molise"],
"Islands": ["Sardinia","Sicily"],
}

In [4]:
colums_to_use = deepcopy(nomimal_columns)
colums_to_use.append("qweek")
colums_to_use.append("region")
colums_to_use.append("endtime")

data = pd.read_csv(
    "https://raw.githubusercontent.com/YouGov-Data/covid-19-tracker/master/data/italy.csv",
    usecols=colums_to_use,
    index_col=["endtime", "qweek", "region"],
    encoding="ISO-8859-1")

In [5]:
data['datetime'] = pd.to_datetime(
    data.index.get_level_values('endtime'), 
    format="%d/%m/%Y %H:%M")

In [6]:
data.reset_index(level='endtime', inplace=True)
data.set_index('datetime', 'qweek', 'region', inplace=True)

In [7]:
for nominal in nominal_to_value:
    data.replace(nominal, nominal_to_value[nominal], inplace=True)

In [8]:
data.replace(' ', 0, inplace=True)

In [9]:
Qweek = {}
qweeks = data.index.get_level_values('qweek').unique()
for qweek in qweeks:
    mean_date = data.loc[qweek,:,:].index.get_level_values('datetime').unique().mean()
    Qweek.update({qweek: mean_date.date()})

# Metodo

In [10]:
display(Markdown(f"""
I dati sono stati (finora) raccolti nell'arco di {qweeks.size} settimane 
a partire dal 2 Aprile 2020 e sono suddivisi 
per aree geografiche: Nord Est, Nord Ovest, Centro, Sud e Isole.
"""))


I dati sono stati (finora) raccolti nell'arco di 20 settimane 
a partire dal 2 Aprile 2020 e sono suddivisi 
per aree geografiche: Nord Est, Nord Ovest, Centro, Sud e Isole.


In [11]:
for zona in zone_regioni:
    regioni = "\n- ".join(zone_regioni[zona])
    subjects = data.loc[data.index.get_level_values("region")==zona].shape[0]
    print(f"{region_ita[zona]} (soggetti: {subjects})")
    print(f"- {regioni}")
    print()

Nord Ovest (soggetti: 5377)
- Aosta
- Liguria
- Lombardy
- Piedmont

Nord Est (soggetti: 3831)
- Emilia-Romagna
- Friuli-Venezia_Giulia
- Trento
- Bolzano
- Veneto

Centro (soggetti: 4019)
- Lazio
- Marche
- Tuscany
- Umbria

Sud (soggetti: 4644)
- Abruzzo
- Apulia
- Basilicata
- Calabria
- Campania
- Molise

Isole (soggetti: 2195)
- Sardinia
- Sicily



In [12]:
lista = "\n- ".join(list(nomimal_columns_dict.values()))
display(Markdown(f"""
Analizzeremo i seguenti risultati:

- {lista}
"""))


Analizzeremo i seguenti risultati:

- Indossare la mascherina all'esterno
- Lavarsi frequentemente le mani
- Usare gel igienizzanti
- Coprirsi naso e bocca se starnuti o tosse
- Evitare contatti con persone sintomatiche
- Evitare di uscire di casa in generale
- Evitare di andare in ospedali e ambulatori
- Evitare di usare i trasporti pubblici
- Evitare di lavorare fuori casa
- Evitare ai figli di andare a scuole e corsi
- Evitare di avere ospiti a casa
- Evitare piccoli assembramenti
- Evitare medi assembramenti
- Evitare grandi assembramenti
- Evitare zone affollate
- Evitare di frequentare negozi
- Dormire in stanze separate
- Mangiare in aree diverse in casa
- Pulire spesso le superfici toccate
- Evitare di toccare oggetti in pubblico


In [13]:
giorni = "\n- ".join(f"{w.replace('week', 'settimana')} = {Qweek[w]}" for w in Qweek)

display(Markdown(f"""
Per ciascuna settimana, i sondaggi sono stati effettuati nell'arco di circa 3 giorni.
Pertanto sceglieremo il giorno medio come indicativo:

- {giorni}
"""))


Per ciascuna settimana, i sondaggi sono stati effettuati nell'arco di circa 3 giorni.
Pertanto sceglieremo il giorno medio come indicativo:

- settimana 1 = 2020-04-03
- settimana 2 = 2020-04-08
- settimana 3 = 2020-04-16
- settimana 4 = 2020-04-25
- settimana 5 = 2020-05-01
- settimana 6 = 2020-05-07
- settimana 7 = 2020-05-14
- settimana 8 = 2020-05-28
- settimana 9 = 2020-06-10
- settimana 10 = 2020-06-25
- settimana 11 = 2020-07-09
- settimana 12 = 2020-07-23
- settimana 13 = 2020-08-07
- settimana 14 = 2020-08-20
- settimana 15 = 2020-09-03
- settimana 16 = 2020-09-20
- settimana 17 = 2020-10-02
- settimana 18 = 2020-10-15
- settimana 19 = 2020-10-29
- settimana 20 = 2020-11-12


In [14]:
risposte = "\n- ".join(f"{nominal_to_value[n]:.2f} = {valori_ita[i]}" for i, n in enumerate(nominal_to_value))

display(Markdown(f"""
Le risposte sono di tipo nominale dal comportamento meno conforme (Mai) al più conforme (Sempre)

- {risposte}
"""))


Le risposte sono di tipo nominale dal comportamento meno conforme (Mai) al più conforme (Sempre)

- 0.00 = Mai
- 0.25 = Raramente
- 0.50 = Qualche volta
- 0.75 = Spesso
- 1.00 = Sempre


# Risultati

## Aree

Mediana e interquartile per ciascuna settimana e area.

In [15]:
md = data.reset_index().melt(id_vars=['qweek', 'region', 'datetime', 'endtime'])

In [16]:
md['variable'] = md.variable.map(nomimal_columns_dict)

In [17]:
md['data'] = md.qweek.map(Qweek)
md['data'] = pd.to_datetime(md.data)

In [18]:
g = sns.relplot(
    kind='line',
    data=md,
    x='data', y='value',
    hue='region',
    col='variable', col_wrap=4,
    #aspect=1, 
    height=4,
)

g.axes.flat[0].set(
    ylim=(0, 1),
    yticks=list(nominal_to_value.values()),
    yticklabels=valori_ita,
)

g.axes.flat[-1].xaxis.set_major_locator(mdates.MonthLocator())
g.axes.flat[-1].xaxis.set_major_formatter(mdates.DateFormatter('%b'))

g.set_titles(col_template="{col_name}")
plt.show()

In [19]:
_ = """zones = ["North east", "North west", "Centre", "South", "Islands"]

fig, ax = plt.subplots(5, 4, figsize=(15, 15), sharex=True, sharey=True)

display_data = {}
for zone in zones:
    display_data.update({zone: {}})
    for nomimal_column in nomimal_columns:
        display_data[zone].update({nomimal_column: []})

i = 0
for nomimal_column in nomimal_columns:
    weeks = []
    #print(f"------> {nomimal_column}")
    #display_data.update({nomimal_column: []})
    for idx in data.index.get_level_values('qweek').unique():
        j = 0
        weeks.append(Qweek[idx])
        for zone in zones:
            week_data = data.loc[idx,zone,:][nomimal_column]
            display_data[zone][nomimal_column].append(week_data.mean())
    for zone in zones:
        #print(f"------> {zone}")
        #print(weeks)
        #print(display_data[zone][nomimal_column])
        ax.flat[i].plot(
            weeks, display_data[zone][nomimal_column],
            lw=1, label=region_ita[zone]
        )
        j += 1
    ax.flat[i].set_title(nomimal_columns_dict[nomimal_column], fontsize=9)
    for x in list(nominal_to_value.values()):
        ax.flat[i].axhline(x, c="k", ls=":", lw=1)
        
    ax.flat[i].fill_between(weeks, 0, .25, color='m', alpha=.1)
    ax.flat[i].fill_between(weeks, .25, .5, color='r', alpha=.1)
    ax.flat[i].fill_between(weeks, .5, .75, color='y', alpha=.1)
    ax.flat[i].fill_between(weeks, .75, 1, color='g', alpha=.1)
    
    i += 1

ax.flat[i-2].legend(loc="lower right", ncol=5, bbox_to_anchor=(1, -.35))
ax[0,0].set_ylim(0, 1)
ax[0,0].set_yticks(list(nominal_to_value.values()))
ax[0,0].set_yticklabels(valori_ita)
ax[0,0].xaxis.set_major_locator(mdates.MonthLocator())
ax[0,0].xaxis.set_major_formatter(mdates.DateFormatter('%b'))
ax[0,0].xaxis.set_minor_locator(mdates.DayLocator())

#plt.subplots_adjust(bottom=0.15)
fig.set_facecolor('w')
plt.show();"""

## Italia

In [20]:
g = sns.relplot(
    kind='line',
    data=md,
    x='data', y='value',
    ci=99,
    col='variable', col_wrap=4,
    #aspect=1, 
    height=4,
)
g.set_titles(col_template="{col_name}")

g.axes.flat[0].set(
    ylim=(0, 1),
    yticks=list(nominal_to_value.values()),
    yticklabels=valori_ita,
)

g.axes.flat[-1].xaxis.set_major_locator(mdates.MonthLocator())
g.axes.flat[-1].xaxis.set_major_formatter(mdates.DateFormatter('%b'))

plt.show()

In [21]:
_ = """fig, ax = plt.subplots(5, 4, figsize=(15, 15), sharex=True, sharey=True)

display_data = {}

i = 0
for nomimal_column in nomimal_columns:
    weeks = []
    display_data.update({nomimal_column: {'mean': [], 'std': []}})
    for idx in data.index.get_level_values('qweek').unique():
        weeks.append(Qweek[idx])
        display_data[nomimal_column]['mean'].append(data.loc[idx,:,:][nomimal_column].mean())
        display_data[nomimal_column]['std'].append(data.loc[idx,:,:][nomimal_column].std())
        
    diff = display_data[nomimal_column]['mean'][-1] - display_data[nomimal_column]['mean'][0]
    if diff <= -.25:
        c="m"
    elif diff <= -.15:
        c="r"
    elif diff <= -.05:
        c="y"
    elif diff > 0:
        c="g"
    else:
        c="b"
    ax.flat[i].plot(
        weeks, np.array(display_data[nomimal_column]['mean']),
        c=c, lw=2
    )
    ax.flat[i].fill_between(
        weeks, 
        np.array(display_data[nomimal_column]['mean'])*(1 - np.array(display_data[nomimal_column]['std'])/2),
        np.array(display_data[nomimal_column]['mean'])*(1 + np.array(display_data[nomimal_column]['std'])/2),
        color=c, alpha=.2
    )
    ax.flat[i].set_title(nomimal_columns_dict[nomimal_column], fontsize=9)
    for x in list(nominal_to_value.values()):
        ax.flat[i].axhline(x, c="k", ls=":", lw=1)
    
    ax.flat[i].fill_between(weeks, 0, .25, color='m', alpha=.1)
    ax.flat[i].fill_between(weeks, .25, .5, color='r', alpha=.1)
    ax.flat[i].fill_between(weeks, .5, .75, color='y', alpha=.1)
    ax.flat[i].fill_between(weeks, .75, 1, color='g', alpha=.1)
    
    i += 1

ax[0,0].set_ylim(0, 1)
ax[0,0].set_yticks(list(nominal_to_value.values()))
ax[0,0].set_yticklabels(valori_ita)
ax[0,0].xaxis.set_major_locator(mdates.MonthLocator())
ax[0,0].xaxis.set_major_formatter(mdates.DateFormatter('%b'))
ax[0,0].xaxis.set_minor_locator(mdates.DayLocator())

fig.set_facecolor('w')
plt.show();"""

## Ultimi dati

In [22]:
order = md[md.data==md.data.max()].groupby('variable').value.mean().sort_values().index

g = sns.catplot(
    kind='bar',
    data=md[md.data==md.data.max()],
    x='value', y='variable',
    order=order,
    ci=99,
    aspect=1.5, height=7,
)

g.axes.flat[0].set(
    #ylim=(0, 1),
    xticks=list(nominal_to_value.values()),
    xticklabels=valori_ita,
    title=f'Ultimi dati aggiornati al {md.data.max()}',
    xlabel=None, ylabel=None
)
plt.show()

In [23]:
_ = """fig, ax = plt.subplots(figsize=(10, 7))

last = data.index.get_level_values('qweek').unique()[-1]

display_last = {}

i = 0
for n, nomimal_column in enumerate(nomimal_columns):
    display_last.update({nomimal_column: {'mean': None, 'std': None}})
    display_last[nomimal_column]['mean'] = data.loc[last,:,:][nomimal_column].mean()
    display_last[nomimal_column]['std'] = data.loc[last,:,:][nomimal_column].std()
        
    ax.scatter(
        display_last[nomimal_column]['mean'], n,
        color="k", marker="o"
    )
    ax.errorbar(
        display_last[nomimal_column]['mean'], n,
        0, display_last[nomimal_column]['std']/2,
        color="k", elinewidth=1, #capsize=3
    )
    ax.axhline(n, c="k", alpha=.1)

for x in list(nominal_to_value.values()):
    ax.axvline(x, c="k", ls=":", lw=1)
    
ax.fill_betweenx([-1,20], 0, .25, color='m', alpha=.1)
ax.fill_betweenx([-1,20], .25, .5, color='r', alpha=.1)
ax.fill_betweenx([-1,20], .5, .75, color='y', alpha=.1)
ax.fill_betweenx([-1,20], .75, 1, color='g', alpha=.1)

ax.set_title(f"Ultimi dati aggiornati al {Qweek[list(Qweek.keys())[-1]]}", fontsize=20)
ax.set_xlim(0, 1)
ax.set_ylim(-1, 20)
ax.set_xticks(list(nominal_to_value.values()))
ax.set_xticklabels(valori_ita)
ax.set_yticks(np.arange(0, len(nomimal_columns)))
ax.set_yticklabels(nomimal_columns_dict.values())
#ax.xaxis.set_major_locator(mdates.MonthLocator())
#ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
#ax.xaxis.set_minor_locator(mdates.DayLocator())

fig.set_facecolor('w')
plt.show();"""

In [24]:
_ = """latest = {}
for param in display_data:
    latest.update({display_data[param]['mean'][-1]: nomimal_columns_dict[param]})


x = 0
fig, ax = plt.subplots(figsize=(10,10))
for p, v in sorted(latest.items()):
    if p > .9:
        c = "g"
    elif p > .75:
        c = "b"
    elif p > .5:
        c = "y"
    elif p > .25:
        c = "r"
    else:
        c = "magenta"
    ax.barh(
        x, p,
        color=c
    )
    _y = .01
    c = "w"
    if p < .5:
        _y = p + .01
        c = "k"
    ax.text(_y, x, v, va="center", ha="left", c=c, fontsize=15)
    x += 1

ax.set_xlim(0, 1)
ax.set_yticks([])
ax.set_xticks(list(nominal_to_value.values()))
ax.set_xticklabels(valori_ita)

ax.set_title(f"Media degli ultimi dati aggiornati al {Qweek[list(Qweek.keys())[-1]]}", fontsize=15)
plt.show();"""

Confronto con i dati della prima settimana:

In [25]:
order = md[md.data==md.data.min()].groupby('variable').value.mean().sort_values().index

g = sns.catplot(
    kind='bar',
    data=md[md.data==md.data.min()],
    x='value', y='variable',
    order=order,
    ci=99,
    aspect=1.5, height=7,
)

g.axes.flat[0].set(
    #ylim=(0, 1),
    xticks=list(nominal_to_value.values()),
    xticklabels=valori_ita,
    title=f'Primi dati del {md.data.min()}',
    xlabel=None, ylabel=None
)
plt.show()

In [26]:
_ = """fig, ax = plt.subplots(figsize=(10, 7))

display_first = {}

i = 0
for n, nomimal_column in enumerate(nomimal_columns):
    display_first.update({nomimal_column: {'mean': None, 'std': None}})
    display_first[nomimal_column]['mean'] = data.loc['week 1',:,:][nomimal_column].mean()
    display_first[nomimal_column]['std'] = data.loc['week 1',:,:][nomimal_column].std()
        
    ax.scatter(
        display_first[nomimal_column]['mean'], n,
        color="k", marker="o"
    )
    ax.errorbar(
        display_first[nomimal_column]['mean'], n,
        0, display_first[nomimal_column]['std']/2,
        color="k", elinewidth=1, #capsize=3
    )
    ax.axhline(n, c="k", alpha=.1)

for x in list(nominal_to_value.values()):
    ax.axvline(x, c="k", ls=":", lw=1)
    
ax.fill_betweenx([-1,20], 0, .25, color='m', alpha=.1)
ax.fill_betweenx([-1,20], .25, .5, color='r', alpha=.1)
ax.fill_betweenx([-1,20], .5, .75, color='y', alpha=.1)
ax.fill_betweenx([-1,20], .75, 1, color='g', alpha=.1)

ax.set_title(f"Primi dati del {Qweek['week 1']}", fontsize=20)
ax.set_xlim(0, 1)
ax.set_ylim(-1, 20)
ax.set_xticks(list(nominal_to_value.values()))
ax.set_xticklabels(valori_ita)
ax.set_yticks(np.arange(0, len(nomimal_columns)))
ax.set_yticklabels(nomimal_columns_dict.values())
#ax.xaxis.set_major_locator(mdates.MonthLocator())
#ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
#ax.xaxis.set_minor_locator(mdates.DayLocator())

fig.set_facecolor('w')
plt.show();"""

In [27]:
_ = """first = {}
for param in display_data:
    first.update({display_data[param]['mean'][0]: nomimal_columns_dict[param]})


x = 0
fig, ax = plt.subplots(figsize=(10,10))
for p, v in sorted(first.items()):
    if p > .9:
        c = "g"
    elif p > .75:
        c = "b"
    elif p > .5:
        c = "y"
    elif p > .25:
        c = "r"
    else:
        c = "magenta"
    ax.barh(
        x, p,
        color=c
    )
    _y = .01
    c = "w"
    if p < .5:
        _y = p + .01
        c = "k"
    ax.text(_y, x, v, va="center", ha="left", c=c, fontsize=15)
    x += 1

ax.set_xlim(0, 1)
ax.set_yticks([])
ax.set_xticks(list(nominal_to_value.values()))
ax.set_xticklabels(valori_ita)

ax.set_title(f"Media dei primi dati del {Qweek['week 1']}", fontsize=15)
plt.show();"""